<div style='background-image: url("../share/Aerial_view_LLNL.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: center ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.8) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">Notebook 2:</div>
            <div style="font-size: x-large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Visualizing Earthquake Magnitudes: <br><br> Amplitude and Distance</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">2014 South Napa Earthquake.</div>
            <div style="font-size: medium ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Estimated Time: 30 minutes.</div>
        </div>
    </div>
</div>



This notebook aims to help you explore seismic data resulting from an earthquake.       
Primarily to visualise the idea of amplitude and distance to help make seismic magnitudes more intuitive.       
In the current example we will explore the 2014 South Napa earthquake. Though, you are welcome to play and explore others.

Topics Covered:
- Obtaining Data through ObsPy
- Plotting Earthquake data with Matplotlib
- Querying Event Data

## Visualizing Earthquake Magnitudes Demo

Now that you have learned some Python basics, you can use your skills to analyze some data. 

This notebook will provide some exercises using [Obspy](https://github.com/obspy/obspy/wiki), a Python library used to visualize and analyze seimsographic data.

In [5]:
# These import a few functions and set the figure size and plot styles for our visualizations

%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 16, 10

### Event Data

The first step is to find the event. Below, we import the Obspy Client which lets us query for earthquake events. Then we get the UTC date and time for the 2014 South Napa earthquake happened (August 24th, 2014). We are getting the events with at least a magnitude of 6.0. After, we are printing the events in our resulting catalog, with a local projection centered on the San Francisco Bay Area.

In [ ]:
import obspy
from obspy.clients.fdsn import Client

client = Client("USGS")
t = obspy.UTCDateTime("2014-08-24T10:20:44.0")  # South Napa earthquake
cat = client.get_events(starttime=t - 100, endtime=t + 3600,
                        minmagnitude=6)

# Print and plot event.
print(cat)
cat.plot(projection="local", resolution="i");
cat.plot().show()

As you can see above, the map is centered on Santa Rosa, showing a 6.0 magnitude event occuring.

---

### Waveform and Station Data

#### Mass Downloader

ObsPy has a submodule to help download data, called mass downloader. It attempts to offer an API for how seismologists would like to download data. It works in three steps:

1. Define Geographical Domain
2. Define Other Restrictions
3. Launch Download

#### Exercise


##### Part A - Download

Use the mass downloader to download data for a small (there are a lot of stations in this region) geographical region (a circular region with a radius of 1 degree is an example), including the station information, with the mass downloader. Download from 2 minutes before the event to 10 minutes after it and download only `BHZ` (Broadband) channels.

First we'll import the libraries needed to download:

In [3]:
import obspy
from obspy.clients.fdsn.mass_downloader import CircularDomain, \
    Restrictions, MassDownloader

The cell below will download:

In [4]:
def download_data():
# First, define a domain.
    domain = CircularDomain(latitude=cat[0].origins[0].latitude,
                        longitude=cat[0].origins[0].longitude,
                        minradius=0.25, maxradius=5.0)

# Second, define some additional restrictions.
    restrictions = Restrictions(starttime=cat[0].origins[0].time - 0.5 * 60,
                            endtime=cat[0].origins[0].time + 5 * 60,
                            minimum_interstation_distance_in_m=100E3,channel="BHZ",)

# If you leave the providers empty it will loop through
# all data centers it knows.
# It will prefer data from the first providers.
    mdl = MassDownloader(providers=["SCEDC", "NCEDC", "IRIS"])

# Finally launch it.
    mdl.download(domain, restrictions,
             mseed_storage="waveforms",
             stationxml_storage="stations")
    
download_data()

NameError: name 'cat' is not defined

##### Part B - Plot Stations

Plot the stations and the event in a single map. 
The code below reads through the stations file and reads the inventory at each file name. Then the inventory is plotted with a local projection and then a title is added to the map.

In [ ]:
import glob

def plot_stations():
    for i,filename in enumerate(glob.glob("stations/*.xml")):
        if i==0:
            inv = obspy.read_inventory(filename)
        else:
            inv+=obspy.read_inventory(filename)

    fig = inv.plot(projection="local", resolution='i',show=False, size=70)
    fig = cat.plot(fig=fig,projection="local", resolution='i',show=False, title='Stations Available for South Napa Quake');
    plt.show()
    
plot_stations()

##### Part C - Process Data

This will filter the data and check that there are no gaps etc.     
Don't worry about this.

In [ ]:
import glob
import os
import numpy as np

inv = obspy.Inventory(networks=[], source="")

# First read all station files.
for filename in glob.glob("./stations/*.xml"):
    inv += obspy.read_inventory(filename)
    
# Now read the waveform files.
st = obspy.read("./waveforms/*.mseed")

st.remove_response(inventory=inv, water_level=60)
st.detrend("linear")
st.taper(max_percentage=0.05)
st.filter("bandpass", freqmin=0.001, freqmax=0.1,
          zerophase=True, corners=6)

max_starttime = max(tr.stats.starttime for tr in st)
min_endtime = min(tr.stats.endtime for tr in st)
npts = int((min_endtime - max_starttime)  / 2.0)

for tr in st:
    tr.data = np.require(tr.data,
                         requirements=["C_CONTIGUOUS"])
    
# Remove instrument response
st.remove_response(inventory=inv)

st.interpolate(sampling_rate=0.5, method="lanczos",
               starttime=max_starttime, npts=npts, a=12)

In [ ]:
import pprint
pprint.pprint("""Possible stations below.
Stations (36):
G01D (WHOI OBS ID S85)
G03D (WHOI OBS ID T111)
G05D (WHOI OBS ID T112)
G21D (WHOI OBS ID T106)
G33D (SIO OBS ID G33D)
M16D (SIO OBS ID M16D)
BDM (Black Diamond Mines Park, Antioch, CA, USA)
BKS (Byerly Seismographic Vault, Berkeley, CA, USA)
CMB (Columbia College, Columbia, CA, USA)
FARB (Farallon Islands, CA, USA)
HATC (Hat Creek Radio Astronomy Obs., Cassel, CA, USA)
HELL (Rademacher Property, Miramonte, CA, USA)
HOPS (Hopland Research and Extension Center, Hopland, CA)
JCC (Jacoby Creek, Bayside, CA, USA)
MOD (Modoc Plateau, CA, USA)
ORV (Oroville Dam, Oroville, CA, USA)
RAMR (Ramage Ranch, Paso Robles, CA, USA)
SAO (San Andreas Geophysical Obs., Hollister, CA, USA)
SUTB (Sutter Buttes, CA, USA)
THIS (South End of Cholame Valley, Shandon, CA, USA)
WDC (Whiskeytown Dam, Whiskeytown, CA, USA)
YBH (Yreka Blue Horn Mine, Yreka, CA, USA)
LRL (Laurel Mtn)
MLAC (Mammoth)
MPP (Mcpherson Peak)
OSI (Osito Audit: Castaic Lake)
SCZ2 (Santa Cruz Island 2)
SPG2 (Springville 2)
NV31 (NVAR Array Site 31, Mina, NV, USA)
K04D (Chiloquin, OR, USA)
L02E (Cave Junction, OR, USA)
M02C (Callahan, CA, USA)
O02D (Mt. Diablo Meridian, CA, USA)
DBO (Dodson Butte, OR, USA (UO CREST BB SMO))
TPNV (Topopah Spring, Nevada, USA)
GB281 (SIO OBS ID GB281))""")

##### Part D - Choose stations and compare amplitudes

**Exercise:** Pick three stations from the above plot.    

       
Try stations:     
1) Roughly in a line and at different distances       
2) Not in a line at the same distance

In [ ]:
# Modify the station labels here before running the cell below this one
StationA = 'BKS'
StationB = 'DBO'
StationC = 'MOD'

In [1]:
fig,ax = plt.subplots(1,1)
Stations = [StationA,StationB,StationC]

# Find maximum amplitude for normailzation
max_amp = 0
for station in Stations: 
    max_amp_temp = max(abs(st.select(station=station)[0].data))
    max_amp = max(max_amp_temp, max_amp)
    
for station in Stations: 
    data = st.select(station=station)[0].data/max_amp# normalize data
    trace = ax.plot(st.select(station=station)[0].times(),data, label=station)
    color = trace[0].get_color()
    ax.axhline(y=max(data), xmin=min(st.select(station=station)[0].times()), xmax=max(st.select(station=station)[0].times()), ls=':', c=color, alpha=0.7)
    ax.axhline(y=min(data), xmin=min(st.select(station=station)[0].times()), xmax=max(st.select(station=station)[0].times()), ls=':', c=color, alpha=0.7)
    

ax.legend()
ax.set_ylim([-1,1])
ax.set_title('Amplitude Comparison')
ax.set_ylabel('Normalizes Amplitude')
ax.set_xlabel('Time [s]')

NameError: name 'plt' is not defined

In [ ]:
def download_data():
    

Notebook developed by Alexander Robson, George Khalilieh

Data Science Modules: http:/data.berkeley.edu/education/modules